# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import re

## Section 1: Web Scraping Postal Codes

In [2]:
#create beautiful soup object for the given url
try:
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 
    request=urllib.request.Request("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",None,headers) #The assembled request
    html = urllib.request.urlopen(request)
except:
    print("Invalid URL, cannot open")
            #return
soup = BeautifulSoup(html, 'lxml')  

In [3]:
#find the table and separate into body and head
table=soup.find_all("table", attrs={"class": "wikitable"})
table1 = table[0]
# the head will form our column names
body = table1.find_all("tr")
# Head values (Column names) are the first items of the body list
head = body[0] # 0th item is the header row
body_rows = body[1:] # All other items becomes the rest of the rows

In [4]:
#load in the headings of the table
headings = []
for item in head.find_all("th"): # loop through all th elements
    # convert the th elements to text and strip "\n"
    item = (item.text).rstrip("\n")
    # append the clean column name to headings
    headings.append(item)
print(headings)

['Postal Code', 'Borough', 'Neighbourhood']


#### Instructions
1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
4. If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
6. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
7. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [5]:
#add in the rows of the table
all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
        row.append(aa) #add to list containing row entry
    if row[1]=='Not assigned': #no borough assigned
        continue;
    else:
        if row[2]=='Not assigned': #borough assigned, but no neighborhood
            row[2]=row[1]
        all_rows.append(row)



In [6]:
df = pd.DataFrame(data=all_rows,columns=headings)

In [7]:
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park Harbourfront
3,M6A,North York,Lawrence Manor Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue Humber Valley Village
6,M1B,Scarborough,Malvern Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill Woodbine Gardens
9,M5B,Downtown Toronto,Garden District Ryerson


In [8]:
df.shape

(103, 3)

## Section 2: Using CSV file to Create Dataframe with Location Data

In [9]:
df_data_1 = pd.read_csv('Geospatial_Coordinates.csv')
df_data_1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df_toronto=pd.merge(df, df_data_1, on='Postal Code', how='outer')
df_toronto.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)
df_toronto.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue Humber Valley Village,43.667856,-79.532242
6,M1B,Scarborough,Malvern Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,Garden District Ryerson,43.657162,-79.378937


## Section 3: Analysis and Clustering of Neighborhoods in Toronto
#### Instructions
You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to 
the New York City data. It is up to you. 
Just make sure:
to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
to generate maps to visualize your neighborhoods and how they cluster together. 

In [11]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [12]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


#### Create initial map of Toronto with all the postal codes marked

In [13]:
toronto_map= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for code,lat, lng, borough, neighborhood in zip(df_toronto['Postal Code'],df_toronto['Latitude'],df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}\n{},{}'.format(code,neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

In [14]:
#load credentials stored in a json file
f = open("foursquare_cred.json") 
cred = json.load(f) 
f.close() 

In [15]:
CLIENT_ID = cred['client_id'] # your Foursquare ID
CLIENT_SECRET = cred['client_secret'] # your Foursquare Secret
ACCESS_TOKEN = cred['access_token'] # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

### First examine a single postal code

In [16]:
df_toronto.loc[df_toronto['Postal Code'] == 'M5T']

,Postal Code,Borough,Neighborhood,Latitude,Longitude
84,M5T,Downtown Toronto,Kensington Market Chinatown Grange Park,43.653206,-79.400049


In [17]:

latitude = df_toronto.loc[84, 'Latitude'] 
longitude = df_toronto.loc[84, 'Longitude'] 
code= df_toronto.loc[84, 'Postal Code'] # postal code

print('Latitude and longitude values of {} are {}, {}.'.format(code, latitude, longitude))

radius=500
LIMIT=100

Latitude and longitude values of M5T are 43.6532057, -79.4000493.


In [18]:
#prepare query url
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6001ba298840934787fc285f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Kensington',
  'headerFullLocation': 'Kensington, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 65,
  'suggestedBounds': {'ne': {'lat': 43.6577057045, 'lng': -79.3938414091248},
   'sw': {'lat': 43.6487056955, 'lng': -79.40625719087521}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b116957f964a520087c23e3',
       'name': 'Kid Icarus',
       'location': {'address': '205 Augusta Ave.',
        'crossStreet': 'Denison Square',
        'lat': 43.653933260442265,
        'lng': -79.40171859012935,
        'labeledLatLngs': [{'label': 'displ

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
#create dataframe of nearby venues
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Kid Icarus,Arts & Crafts Store,43.653933,-79.401719
1,Seven Lives - Tacos y Mariscos,Mexican Restaurant,43.654418,-79.400545
2,Essence of Life Organics,Organic Grocery,43.654111,-79.400431
3,Blackbird Baking Co,Bakery,43.654897,-79.400619
4,The Moonbean Cafe,Café,43.654147,-79.400182


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

65 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500): #function for extracting venue info from many locations
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Get venue info from all Toronto neighborhoods

In [24]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude'])
print(toronto_venues.shape)
toronto_venues.head()

Parkwoods
Victoria Village
Regent Park Harbourfront
Lawrence Manor Lawrence Heights
Queen's Park Ontario Provincial Government
Islington Avenue Humber Valley Village
Malvern Rouge
Don Mills
Parkview Hill Woodbine Gardens
Garden District Ryerson
Glencairn
West Deane Park Princess Gardens Martin Grove Islington Cloverdale
Rouge Hill Port Union Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate Bloordale Gardens Old Burnhamthorpe Markland Wood
Guildwood Morningside West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor Wilson Heights Downsview North
Thorncliffe Park
Richmond Adelaide King
Dufferin Dovercourt Village
Scarborough Village
Fairview Henry Farm Oriole
Northwood Park York University
East Toronto Broadview North (Old East York)
Harbourfront East Union Station Toronto Islands
Little Portugal Trinity
Kennedy Park Ionview East Birchmount Park
Bayview Village
Downsvi

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
Alderwood Long Branch,8,8,8,8,8,8
Bathurst Manor Wilson Heights Downsview North,23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
Bedford Park Lawrence Manor East,24,24,24,24,24,24
Berczy Park,58,58,58,58,58,58
Birch Cliff Cliffside West,4,4,4,4,4,4
Brockton Parkdale Village Exhibition Place,23,23,23,23,23,23
Business reply mail Processing Centre South Central Letter Processing Plant Toronto,17,17,17,17,17,17


In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alderwood Long Branch,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bathurst Manor Wilson Heights Downsview North,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bedford Park Lawrence Manor East,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Birch Cliff Cliffside West,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Brockton Parkdale Village Exhibition Place,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Business reply mail Processing Centre South Ce...,0.058824,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,CN Tower King and Spadina Railway Lands Harbou...,0.000000,0.0,0.071429,0.071429,0.142857,0.214286,0.142857,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Ethiopian Restaurant,Escape Room,Event Space,Electronics Store,Diner
1,Alderwood Long Branch,Pizza Place,Pharmacy,Sandwich Place,Coffee Shop,Pool,Pub,Gym,Airport Terminal,Farmers Market,Event Space
2,Bathurst Manor Wilson Heights Downsview North,Bank,Coffee Shop,Grocery Store,Restaurant,Supermarket,Sushi Restaurant,Intersection,Deli / Bodega,Bridal Shop,Ice Cream Shop
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store
4,Bedford Park Lawrence Manor East,Thai Restaurant,Sandwich Place,Italian Restaurant,Coffee Shop,Greek Restaurant,Comfort Food Restaurant,Butcher,Juice Bar,Café,Japanese Restaurant
5,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Cheese Shop,Farmers Market,Seafood Restaurant,Beer Bar,Japanese Restaurant,Clothing Store
6,Birch Cliff Cliffside West,College Stadium,Skating Rink,General Entertainment,Café,Escape Room,Electronics Store,Eastern European Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Dim Sum Restaurant
7,Brockton Parkdale Village Exhibition Place,Café,Breakfast Spot,Coffee Shop,Bakery,Burrito Place,Stadium,Italian Restaurant,Bar,Restaurant,Furniture / Home Store
8,Business reply mail Processing Centre South Ce...,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Park,Pizza Place
9,CN Tower King and Spadina Railway Lands Harbou...,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Rental Car Location,Airport,Airport Food Court,Boat or Ferry,Harbor / Marina,Coffee Shop


#### Group neighborhoods into 5 clusters

In [46]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 3, 3, 3, 3, 3, 3, 3, 3])

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [48]:
toronto_merged = df_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Food & Drink Shop,Park,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,Intersection,Pizza Place,French Restaurant,Portuguese Restaurant,Hockey Arena,Coffee Shop,Donut Shop,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,Regent Park Harbourfront,43.654260,-79.360636,3.0,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Restaurant,Theater,Yoga Studio,Brewery
3,M6A,North York,Lawrence Manor Lawrence Heights,43.718518,-79.464763,3.0,Clothing Store,Furniture / Home Store,Boutique,Accessories Store,Event Space,Vietnamese Restaurant,Sporting Goods Shop,Coffee Shop,Drugstore,Distribution Center
4,M7A,Downtown Toronto,Queen's Park Ontario Provincial Government,43.662301,-79.389494,3.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Café,Italian Restaurant,Japanese Restaurant,Beer Bar,Fast Food Restaurant,Smoothie Shop,Burrito Place


In [49]:
toronto_merged.dropna(inplace=True) #drop any rows with NaN

#### Create map containing all the clusters

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0,toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0.0,Food & Drink Shop,Park,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
21,York,0.0,Park,Women's Store,Bar,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
35,East York,0.0,Park,Convenience Store,Metro Station,Women's Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
64,York,0.0,Park,Convenience Store,Jewelry Store,Women's Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
66,North York,0.0,Park,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
85,Scarborough,0.0,Intersection,Playground,Park,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
91,Downtown Toronto,0.0,Park,Trail,Playground,Women's Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


Cluster 0, denoted by the red markers on the map, consists of 7 neighborhoods. This cluster is best characterized by its high frequency of outdoor recreational spaces, with the majority of neighborhoods having parks as their most common venue, in addition to having dog runs as well. 

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,North York,1.0,Pizza Place,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Harbor / Marina
62,Central Toronto,1.0,Home Service,Garden,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
63,York,1.0,Pizza Place,Grocery Store,Brewery,Convenience Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
72,North York,1.0,Pizza Place,Grocery Store,Coffee Shop,Butcher,Pharmacy,Greek Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
89,Etobicoke,1.0,Pizza Place,Grocery Store,Pharmacy,Fried Chicken Joint,Sandwich Place,Beer Store,Fast Food Restaurant,Video Store,Golf Course,Deli / Bodega
93,Etobicoke,1.0,Pizza Place,Pharmacy,Sandwich Place,Coffee Shop,Pool,Pub,Gym,Airport Terminal,Farmers Market,Event Space


Cluster 1, denoted by the 6 spread out purple markers on the map, can best be characterized by neighborhoods with a high frequency of pizza place restaurants. Additionally, common amenities such as grocery stores, pharmacies, and drugstores can be found among these neighborhoods as well.

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,2.0,Filipino Restaurant,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Fast Food Restaurant


Cluster 2, denoted by the single blue marker on the map, is a neighborhood that contains a heterogeneous mixure of venues that don't quite make it fit in with other neighborhoods. 

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3,toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,3.0,Intersection,Pizza Place,French Restaurant,Portuguese Restaurant,Hockey Arena,Coffee Shop,Donut Shop,Diner,Discount Store,Distribution Center
2,Downtown Toronto,3.0,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Restaurant,Theater,Yoga Studio,Brewery
3,North York,3.0,Clothing Store,Furniture / Home Store,Boutique,Accessories Store,Event Space,Vietnamese Restaurant,Sporting Goods Shop,Coffee Shop,Drugstore,Distribution Center
4,Downtown Toronto,3.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Café,Italian Restaurant,Japanese Restaurant,Beer Bar,Fast Food Restaurant,Smoothie Shop,Burrito Place
7,North York,3.0,Gym,Beer Store,Japanese Restaurant,Coffee Shop,Restaurant,Dim Sum Restaurant,Bike Shop,Sporting Goods Shop,Sandwich Place,Café
8,East York,3.0,Pizza Place,Athletics & Sports,Gastropub,Intersection,Flea Market,Pharmacy,Café,Breakfast Spot,Bank,Gym / Fitness Center
9,Downtown Toronto,3.0,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Bubble Tea Shop,Cosmetics Shop,Hotel,Japanese Restaurant,Lingerie Store,Pizza Place
10,North York,3.0,Park,Metro Station,Japanese Restaurant,Asian Restaurant,Pub,Pizza Place,Bakery,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
12,Scarborough,3.0,Bar,Women's Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field
13,North York,3.0,Gym,Beer Store,Japanese Restaurant,Coffee Shop,Restaurant,Dim Sum Restaurant,Bike Shop,Sporting Goods Shop,Sandwich Place,Café


Cluster 3, denoted by the numerous mint colored markers of the map, contains neighborhoods of great diversity, the majority of which are concentrated in downtown Toronto. The most common venue of the neighborhoods, at a quick glance, seems to be coffee shops. Also due to the number of neighborhoods in this cluster, there is a diverse variety of restaurants and eateries, in addition to entertainment and shopping places. 

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4,toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,4.0,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Wings Joint
56,York,4.0,Fast Food Restaurant,Restaurant,Discount Store,Fried Chicken Joint,Sandwich Place,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dim Sum Restaurant


Cluster 4, denoted by the two orange markers on the map, consists of 2 two neighborhoods which are very similar to each other. They each have fast food restaurant as most common venue, and have dumpling restaurant, discount store, eastern european restaurant, and drugstore in their top 10 most common venues.

#### Cluster Summary
Cluster 0: The Great Outdoors
Cluster 1: Number 1 Pizza 
Cluster 2: The Standout Little Bit of Everything
Cluster 3: Urban Life at its Best
Cluster 4: A Pair of Fast Foodies